In [ ]:
# Library yang akan dipakai
import zipfile
import os
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, AveragePooling2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers.core import Dense, Dropout, Flatten

# Untuk load data
from imutils import paths
from PIL import Image
import argparse
import matplotlib.pyplot as plt # untuk menampilkan contoh gambar/data dalam gambar

# Untuk Data Loading -> yang membagi data
from sklearn.preprocessing import LabelBinarizer # Untuk binarisasi label
from sklearn.model_selection import train_test_split # Untuk ngesplit data train & test
from sklearn.metrics import classification_report # Untuk performance measurement, untuk menghasilkan confusion matriks

In [ ]:
# Import data from GDrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Extract the zip file

local_zip = '/content/gdrive/MyDrive/Colab Notebooks/FGA/ML to DL Image Classification/Copy of 3scenes.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/gdrive/MyDrive/Colab Notebooks/FGA/ML to DL Image Classification/')
zip_ref.close()

KeyboardInterrupt: ignored

In [ ]:
# Some stuffs from Dicoding

from tensorflow.keras.preprocessing.image import ImageDataGenerator

total_gen = ImageDataGenerator(
                    validation_split=0.25,
                    rescale=1./255)

train_gen = total_gen
train_gen.horizontal_flip = True
train_gen.fill_mode = "nearest"

train_generator = train_gen.flow_from_directory(
                    '/content/gdrive/MyDrive/Colab Notebooks/FGA/ML to DL Image Classification/3scenes',
                    target_size = (32,32),
                    batch_size = 4,
                    class_mode = 'categorical',
                    subset = 'training'                    
                    )
validation_generator = total_gen.flow_from_directory(
                    '/content/gdrive/MyDrive/Colab Notebooks/FGA/ML to DL Image Classification/3scenes',
                    target_size = (32,32),
                    batch_size = 4,
                    class_mode = 'categorical',
                    subset = 'validation'  
                    )


Found 711 images belonging to 3 classes.
Found 237 images belonging to 3 classes.


In [ ]:
model = Sequential()
model.add(Conv2D(16, kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'relu', input_shape = (32,32 ,3)))
model.add(AveragePooling2D(2,2))
model.add(BatchNormalization())


model.add(Conv2D(32, kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'relu'))
model.add(AveragePooling2D(2,2))
model.add(BatchNormalization())


model.add(Conv2D(32, kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'relu'))
model.add(AveragePooling2D(2,2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(3, activation = 'softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 16)        448       
_________________________________________________________________
average_pooling2d_4 (Average (None, 16, 16, 16)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 16, 16, 16)        64        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 32)        4640      
_________________________________________________________________
average_pooling2d_5 (Average (None, 8, 8, 32)          0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 8, 8, 32)          128       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 8, 32)         

In [ ]:
# compile model dengan 'adam' optimizer loss function 'categorical_crossentropy' 
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(learning_rate=1e-3, decay=1e-3 / 50),
              metrics=['accuracy'])

In [ ]:
model.fit(
      train_generator,
      steps_per_epoch=25,  
      epochs=50, 
      validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
      validation_steps=5,
      verbose = 1)

Epoch 1/50
25/25 [==============================] - 34s 1s/step - loss: 1.7503 - accuracy: 0.4200 - val_loss: 1.0811 - val_accuracy: 0.5500
Epoch 2/50
25/25 [==============================] - 29s 1s/step - loss: 0.8839 - accuracy: 0.6400 - val_loss: 1.0451 - val_accuracy: 0.5500
Epoch 3/50
25/25 [==============================] - 24s 968ms/step - loss: 0.7889 - accuracy: 0.6400 - val_loss: 1.0813 - val_accuracy: 0.5000
Epoch 4/50
25/25 [==============================] - 19s 752ms/step - loss: 0.5422 - accuracy: 0.7800 - val_loss: 1.0451 - val_accuracy: 0.4000
Epoch 5/50
25/25 [==============================] - 19s 777ms/step - loss: 0.6897 - accuracy: 0.7300 - val_loss: 1.0888 - val_accuracy: 0.4500
Epoch 6/50
25/25 [==============================] - 15s 601ms/step - loss: 0.4319 - accuracy: 0.8500 - val_loss: 0.8673 - val_accuracy: 0.6000
Epoch 7/50
25/25 [==============================] - 13s 532ms/step - loss: 0.5101 - accuracy: 0.7900 - val_loss: 1.2464 - val_accuracy: 0.3500
Epoch

Another way to Load Data in zipfile.

**Development Case**


Cari tau implemenetasi dalam pebuatan model gimana caranya.
1. Menggunakan Notebook(bisa dianggap sebagai terminal, bisa dieksekusi baris perbaris) 
    Training dan Testing terdapat dalam 1 notebook(ipynb)

    Notebook, Google Colab, IBM Watson, dll (Kebanyakan skrg file .ipynb)

2. File Python
    Untuk membuat untuk training dan testing dipisah (ex : training.py & testing.py)

    Dirun dalam terminal(bisa lewat terminal/notebook) .py

**Data Loading**
Ada macam2 :
1. Membaca data
2. Membagi data menjadi data training dan data testing

Data training bisa dibagi menjadi 2 , training & validation -> Data yang dipakai untuk proses training, dipakai untuk mengupdate model, weight.

Data testing -> Dipakai untuk melihat performa dari model. Data ini belum pernah dipakai/dilihat oleh model

Problem -> Gimana cara membagi datanya?
1. Membagi data menjadi 2 folder terpisah untuk training dan testing, kalau belum?
2. Cek di file bapanya https://colab.research.google.com/drive/1kYfi4njzcaQBx-18bpa-K5vcm5woQYJQ?usp=sharing#scrollTo=XQNvMZsuzZtp


trainX, testX, trainY, testY

X = gambar, Y = label

**Cek juga drive bapanya**

https://drive.google.com/drive/folders/18Y6NI8nN7Iu5wM8nnlUW0GFDlc0EWo_r?usp=sharing


20/08/2021

Didalam Deep Learning dibagi menjadi 2 bagian

Feature Extraction Layer

Pertama -> Extract Input, Pooling ,dsb

Didalam Dense, ada 3 output(misal ada 3 kelas)

RBG-D/RBG-T(RGB with thermal) Object Recognition using MultiModal Deep Neural Network(Search this Shit on Google)

Object Detection -> pake image MS-COCO, suatu dataset, jumlah jutaan, udah ada pretrain weightnya juga. Terdapat 80 kelas

Kita perlu tahu filosofi dari setiap setting dan di modifikasi isi modelnya



